In [2]:
# importing all required packages & notebook extensions at the start of the notebook
import os
import biom
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization
%matplotlib inline

In [3]:
# assigning variables throughout the notebook
raw_data_dir = "../data/raw"
data_dir = "../data/processed"
vis_dir  = "../results"

In [4]:
Visualization.load(f"{data_dir}/core-metrics-results-bt/unweighted_unifrac_emperor.qzv")

<visualization: Visualization uuid: 41d418f4-336b-4036-a276-6c87940060b8>

In [5]:
Visualization.load(f"{data_dir}/core-metrics-results-bt/weighted_unifrac_emperor.qzv")

<visualization: Visualization uuid: cbbe3e41-299a-41e3-a6f6-8f5bc217a8fe>

In [6]:
Visualization.load(f"{data_dir}/core-metrics-results-bt/bray_curtis_emperor.qzv")

<visualization: Visualization uuid: cbffe7f4-4a6d-4357-a60e-f85724dafb47>

In [7]:
! qiime diversity adonis \
--i-distance-matrix $data_dir/core-metrics-results-bt/weighted_unifrac_distance_matrix.qza \
--m-metadata-file $data_dir/metadata.tsv \
--p-formula Cohort_Number*Stool_Consistency*Patient_Sex \
--o-visualization $data_dir/core-metrics-results-bt/weighted_unifrac_adonis.qzv

Saved Visualization to: ../data/processed/core-metrics-results-bt/weighted_unifrac_adonis.qzv


In [8]:
Visualization.load(f"{data_dir}/core-metrics-results-bt/weighted_unifrac_adonis.qzv")

<visualization: Visualization uuid: 2a0d9c2d-f6b0-451a-828f-ed50f6ec2311>

In [9]:
metadata = pd.read_csv(f"{data_dir}/metadata.tsv", sep='\t')
metadata['Cohort_Number_Bin'] = metadata['Cohort_Number'].map({1: 'Abduction', 2: 'Recovery'})
metadata.to_csv(f"{data_dir}/metadata_binned.tsv", sep='\t', index=False)

In [10]:
! qiime diversity beta-group-significance \
    --i-distance-matrix $data_dir/core-metrics-results-bt/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file $data_dir/metadata_binned.tsv \
    --m-metadata-column Cohort_Number_Bin \
    --p-pairwise \
    --o-visualization $data_dir/core-metrics-results-bt/weighted_unifrac-Cohort-number-significance.qzv

Saved Visualization to: ../data/processed/core-metrics-results-bt/weighted_unifrac-Cohort-number-significance.qzv


In [11]:
Visualization.load(f"{data_dir}/core-metrics-results-bt/weighted_unifrac-Cohort-number-significance.qzv")

<visualization: Visualization uuid: f363fd1f-256e-45db-a3c2-8fab452b3124>